In [49]:
import pandas as pd
import numpy as np
import os
import json
import requests
from pprint import pprint

In [50]:
confirmed_df = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
us = confirmed_df.loc[confirmed_df['Country/Region']=='US']
confirmed_df.drop(us.index[0], axis=0, inplace=True)
confirmed_df.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,120,170,174,237,273,281,299,349,367,423
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,212,223,243,259,277,304,333,361,377,383
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,511,584,716,847,986,1171,1251,1320,1423,1468
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,334,370,376,390,428,439,466,501,525,545
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,7,7,7,8,8,8,10,14,16,17


In [51]:
confirmed_us_df = pd.read_csv('https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
confirmed_us_df = confirmed_us_df.iloc[:,6:]
confirmed_us_df.drop(['Combined_Key'], axis=1, inplace=True)
confirmed_us_df.rename(columns={'Province_State': 'Province/State',
                               'Country_Region': 'Country/Region',
                               'Long_': 'Long'}, inplace=True)
print(len(confirmed_us_df))
confirmed_us_df.head()

3253


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
0,American Samoa,US,-14.2710,-170.1320,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Guam,US,13.4443,144.7937,0,0,0,0,0,0,...,56,58,69,77,82,84,93,112,113,121
2,Northern Mariana Islands,US,15.0979,145.6739,0,0,0,0,0,0,...,0,0,2,6,6,6,6,6,6,6
3,Puerto Rico,US,18.2208,-66.5901,0,0,0,0,0,0,...,127,174,239,286,316,316,452,475,513,573
4,Virgin Islands,US,18.3358,-64.8963,0,0,0,0,0,0,...,23,30,30,30,30,37,40,42,43,43


In [52]:
# Drop rows with zero cases
last_column = confirmed_us_df[confirmed_us_df.columns[-1]]
rows = confirmed_us_df.loc[last_column == 0].index
confirmed_us_df.drop(rows, inplace=True)

# Drop cruise ship with no location
cruises = confirmed_us_df.loc[confirmed_us_df['Province/State'].str.contains('Grand Princess')].index
confirmed_us_df.drop(cruises, inplace=True)

print(len(confirmed_us_df))
confirmed_us_df.tail()

2534


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
3231,New Jersey,US,0.0,0.0,0,0,0,0,0,0,...,3020,3847,3686,4512,4866,4808,3935,3821,3521,2207
3240,Rhode Island,US,0.0,0.0,0,0,0,0,0,0,...,57,171,184,160,80,146,241,238,332,353
3243,Tennessee,US,0.0,0.0,0,0,0,0,0,0,...,190,197,307,442,6,54,63,86,69,59
3246,Vermont,US,0.0,0.0,0,0,0,0,0,0,...,7,28,11,14,5,6,11,13,21,3
3248,Washington,US,0.0,0.0,0,0,0,0,0,0,...,125,274,274,303,344,501,483,533,648,677


In [53]:
def lat_long(state):

    url = "https://google-maps-geocoding.p.rapidapi.com/geocode/json"

    querystring = {"language":"en","address":state}

    headers = {
        'x-rapidapi-host': "google-maps-geocoding.p.rapidapi.com",
        'x-rapidapi-key': GMAPS_API_KEY
        }

    response = requests.get(url, headers=headers, params=querystring).json()

    location = response['results'][0]['geometry']['location']
    lat = location['lat']
    long = location['lng']

    return lat, long

In [54]:
for index, row in confirmed_us_df.iterrows():
    state = row['Province/State']
    if ((row['Lat']==0) and (row['Long']==0) and (row['Country/Region']=='US')):
#         try:
        lat, long = lat_long(state)
        confirmed_us_df.loc[index, 'Lat'] = lat
        confirmed_us_df.loc[index, 'Long'] = long
#         except (IndexError, KeyError): 
#             print(index, state)
#             confirmed_us_df.drop(row, inplace=True)
#             pass

print(len(confirmed_us_df))
confirmed_us_df

2534


,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20
1,Guam,US,13.444300,144.793700,0,0,0,0,0,0,...,56,58,69,77,82,84,93,112,113,121
2,Northern Mariana Islands,US,15.097900,145.673900,0,0,0,0,0,0,...,0,0,2,6,6,6,6,6,6,6
3,Puerto Rico,US,18.220800,-66.590100,0,0,0,0,0,0,...,127,174,239,286,316,316,452,475,513,573
4,Virgin Islands,US,18.335800,-64.896300,0,0,0,0,0,0,...,23,30,30,30,30,37,40,42,43,43
5,Alabama,US,32.539527,-86.644082,0,0,0,0,0,0,...,6,6,7,8,10,12,12,12,12,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3231,New Jersey,US,40.058324,-74.405661,0,0,0,0,0,0,...,3020,3847,3686,4512,4866,4808,3935,3821,3521,2207
3240,Rhode Island,US,41.580095,-71.477429,0,0,0,0,0,0,...,57,171,184,160,80,146,241,238,332,353
3243,Tennessee,US,35.517491,-86.580447,0,0,0,0,0,0,...,190,197,307,442,6,54,63,86,69,59
3246,Vermont,US,44.558803,-72.577841,0,0,0,0,0,0,...,7,28,11,14,5,6,11,13,21,3


In [55]:
combined_df = pd.concat([confirmed_df, confirmed_us_df], sort=False)
combined_df.drop(combined_df.columns[4:-1], axis=1, inplace=True)
combined_df.head()

,Province/State,Country/Region,Lat,Long,4/7/20
0,NaN,Afghanistan,33.0000,65.0000,423
1,NaN,Albania,41.1533,20.1683,383
2,NaN,Algeria,28.0339,1.6596,1468
3,NaN,Andorra,42.5063,1.5218,545
4,NaN,Angola,-11.2027,17.8739,17


In [56]:
file = os.path.join('confirmed.csv')
combined_df.to_csv(file, index=False, header=True)